### Basic Overview

We will be exploring random forest methods to build a predictive model for predicting housing prices, given relevant data.

As detailed in https://roamanalytics.com/2016/10/28/are-categorical-variables-getting-lost-in-your-random-forests/, we 
will be using h2o instead of sklearn as that is more tailored to handling of  categorical variables.

In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../../common_routines/')
from relevant_functions import\
    evaluate_model_score_given_predictions,\
    evaluate_model_score


#### Get clean data first

In [2]:
train_data = pd.read_csv('../../cleaned_input/train_data.csv')
validation_data = pd.read_csv('../../cleaned_input/validation_data.csv')

In [3]:
train_data.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotArea', 'Street', 'LotShape',
       'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       ...
       'LogGarageArea', 'LogWoodDeckSF', 'LogOpenPorchSF', 'LogEnclosedPorch',
       'Log3SsnPorch', 'LogScreenPorch', 'LogPoolArea', 'LogMiscVal',
       'LogSalePrice', 'LogMasVnrArea_times_not_missing'],
      dtype='object', length=102)

In [4]:
train_validation_data = pd.concat([train_data, validation_data])

In [5]:
test_data = pd.read_csv('../../input/test.csv')

In [6]:
test_data.isnull().sum().sum()

7000

In [7]:
## Are they indeed clean ?
train_data.isnull().sum().any()

False

In [10]:
validation_data.isnull().sum().any()

False

#### Get h2o up and running !


In [11]:
# Using h2o
import h2o
h2o.init(nthreads = -1, max_mem_size = 15)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_121"; OpenJDK Runtime Environment (Zulu 8.20.0.5-macosx) (build 1.8.0_121-b15); OpenJDK 64-Bit Server VM (Zulu 8.20.0.5-macosx) (build 25.121-b15, mixed mode)
  Starting server from /Users/babs4JESUS/anaconda3/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/cz/3nvpl4mj0g5ds3hlsc15wxdr0000gn/T/tmp_6smdstl
  JVM stdout: /var/folders/cz/3nvpl4mj0g5ds3hlsc15wxdr0000gn/T/tmp_6smdstl/h2o_babs4JESUS_started_from_python.out
  JVM stderr: /var/folders/cz/3nvpl4mj0g5ds3hlsc15wxdr0000gn/T/tmp_6smdstl/h2o_babs4JESUS_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.1
H2O cluster version age:,8 days
H2O cluster name:,H2O_from_python_babs4JESUS_zinejs
H2O cluster total nodes:,1
H2O cluster free memory:,13.33 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


### Brief framework.

We will be building according the following framework (similar to how we did for PCA).

Given set of columns, we should be able to do the following :

1. Train model on training set.

2. Validate on validation set.

3. Generate predicitons on test data.

4. Do cross validation on combined set of training/validation data.


In [12]:
ALL_CATEGORICAL_COLUMNS = ['MSSubClass',
 'MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'MoSold',
 'YrSold',
 'SaleType',
 'SaleCondition']

In [13]:
ALL_NUMERICAL_COLUMNS = ['LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea_times_not_missing',
 'MasVnrArea_not_missing',
 'BsmtFinSF1',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageYrBlt_times_not_missing',
 'GarageYrBlt_not_missing',
 'GarageCars',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal']

In [14]:
ALL_COLUMNS = ALL_CATEGORICAL_COLUMNS + ALL_NUMERICAL_COLUMNS

In [15]:
# Columns the model to be trained 
# Check out ExterQual and YearBuilt instead of YearRemodAdd
# Check out BsmtCond
#cat_cols_in_model = ['MSSubClass', 'Neighborhood', 'ExterQual', 'Foundation', 'BsmtQual', 'BsmtCond',
#                     'BsmtFinType1']
#numeric_cols_in_model = ['GrLivArea', 'OverallQual', 'OverallCond', 'YearRemodAdd', 'BsmtFinSF1', 
#                         'TotalBsmtSF']
# Check out GarageCond
cat_cols_in_model = ['MSSubClass', 'Neighborhood']
numeric_cols_in_model = ['GrLivArea', 'OverallQual', 'OverallCond', 'YearRemodAdd', 'BsmtFinSF1', 
                         'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GarageArea', 'LowQualFinSF']

all_cols_in_model = cat_cols_in_model + numeric_cols_in_model

dep_var_col = 'LogSalePrice'

In [16]:
all_cols_in_model

['MSSubClass',
 'Neighborhood',
 'GrLivArea',
 'OverallQual',
 'OverallCond',
 'YearRemodAdd',
 'BsmtFinSF1',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'GarageArea',
 'LowQualFinSF']

#### Training model on the training set


In [17]:
def get_h2o_frame_with_rel_factors(test_data):
    test_data_h2o = h2o.H2OFrame(test_data)
    for col in ALL_CATEGORICAL_COLUMNS:
        test_data_h2o[col] = test_data_h2o[col].asfactor()
    return test_data_h2o

In [18]:
from h2o.estimators.random_forest import H2ORandomForestEstimator

In [19]:
hpr_1 = H2ORandomForestEstimator(model_id='housing_price_regression', seed=1)
hpr_1.train(x=all_cols_in_model, 
            y=dep_var_col, 
            training_frame=get_h2o_frame_with_rel_factors(train_data))

Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%


In [20]:
predict_out = hpr_1.predict(
    get_h2o_frame_with_rel_factors(train_data))                
train_data['Predictions'] = predict_out.as_data_frame()['predict'].values.tolist()

Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%


In [21]:
#evaluate_model_score_given_predictions(np.log(train_data['Predictions'].values), 
#                                       np.log(train_data[dep_var_col].values))

In [22]:
evaluate_model_score_given_predictions((train_data['Predictions'].values), 
                                       (train_data[dep_var_col].values))

0.05519499883354487

#### Inspect the output model

It may seem like a trivial thing, but shouldn't we inspect the model to see what exactly it does ? This is especially important in data science and it is very easy to get entangled in a quagmire of models and functions without clearly understanding what any of them does.


In [23]:
from h2o.tree import H2OTree
tree = H2OTree(model = hpr_1, tree_number = 0, tree_class = None)

In [24]:
tree

In [25]:
len(tree)

1349

In [26]:
print(tree)

Tree related to model housing_price_regression. Tree number is 0, tree class is 'None'




In [27]:
tree.levels

[None,
 ['Blueste',
  'BrDale',
  'BrkSide',
  'Edwards',
  'IDOTRR',
  'MeadowV',
  'Mitchel',
  'NAmes',
  'NPkVill',
  'OldTown',
  'SWISU',
  'Sawyer'],
 ['Blmngtn',
  'ClearCr',
  'CollgCr',
  'Crawfor',
  'Gilbert',
  'NWAmes',
  'NoRidge',
  'NridgHt',
  'SawyerW',
  'Somerst',
  'StoneBr',
  'Timber',
  'Veenker'],
 None,
 None,
 None,
 None,
 ['30', '40', '50', '70', '75', '160', '180', '190'],
 ['20', '45', '60', '80', '85', '90', '120'],
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 ['30', '40', '70', '160', '180'],
 ['50', '75', '190'],
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 ['20',
  '30',
  '40',
  '45',
  '50',
  '75',
  '80',
  '85',
  '90',
  '120',
  '160',
  '180',
  '190'],
 ['60', '70'],
 None,
 None,
 None,
 None,
 None,
 None,
 ['Blmngtn',
  'ClearCr',
  'CollgCr',
  'Gilbert',
  'NWAmes',
  'SawyerW',
  'Timber

In [28]:
tree.tree_number

0

In [29]:
tree.show()

Tree related to model housing_price_regression. Tree number is 0, tree class is 'None'




In [30]:
print(tree.root_node)

Node ID 0 
Left child node ID = 1
Right child node ID = 2

Splits on column Neighborhood
  - Categorical levels going to the left node: ['Blueste', 'BrDale', 'BrkSide', 'Edwards', 'IDOTRR', 'MeadowV', 'Mitchel', 'NAmes', 'NPkVill', 'OldTown', 'SWISU', 'Sawyer']
  - Categorical levels going to the right node: ['Blmngtn', 'ClearCr', 'CollgCr', 'Crawfor', 'Gilbert', 'NWAmes', 'NoRidge', 'NridgHt', 'SawyerW', 'Somerst', 'StoneBr', 'Timber', 'Veenker']

NA values go to the RIGHT


In [31]:
print(tree.root_node.left_child)

Node ID 1 
Left child node ID = 3
Right child node ID = 4

Splits on column GrLivArea
Split threshold < 1377.5 to the left node, >= 1377.5 to the right node 

NA values go to the LEFT


#### Testing model on validation set

In [32]:
predict_out = hpr_1.predict(
    get_h2o_frame_with_rel_factors(validation_data))                
validation_data['Predictions'] = predict_out.as_data_frame()['predict'].values.tolist()

Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%


In [33]:
#evaluate_model_score_given_predictions(np.log(validation_data['Predictions'].values), 
#                                       np.log(validation_data[dep_var_col].values))

In [34]:
validation_score = evaluate_model_score_given_predictions((validation_data['Predictions'].values), 
                                                          (validation_data[dep_var_col].values))
print(validation_score)

0.13821357736206205


#### Generate predictions on test data

In [35]:
test_data_one_hot = pd.read_csv('../../cleaned_input/test_data_one_hot.csv')

In [36]:
test_data['LogMasVnrArea_times_not_missing'] = test_data_one_hot['LogMasVnrArea_times_not_missing']

In [37]:
test_data[all_cols_in_model].isnull().sum()

MSSubClass      0
Neighborhood    0
GrLivArea       0
OverallQual     0
OverallCond     0
YearRemodAdd    0
BsmtFinSF1      1
TotalBsmtSF     1
1stFlrSF        0
2ndFlrSF        0
GarageArea      1
LowQualFinSF    0
dtype: int64

In [38]:
hpr_1 = H2ORandomForestEstimator(model_id='housing_price_regression', seed=1)
hpr_1.train(x=all_cols_in_model, 
            y=dep_var_col, 
            training_frame=get_h2o_frame_with_rel_factors(train_validation_data))

Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%


In [39]:
predict_out = hpr_1.predict(
    get_h2o_frame_with_rel_factors(test_data))                
test_data['Predictions'] = predict_out.as_data_frame()['predict'].values.tolist()

Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%


/Users/babs4JESUS/anaconda3/lib/python3.6/site-packages/h2o/job.py:69: UserWarning: Test/Validation dataset column 'MSSubClass' has levels not trained on: [150]
  warnings.warn(w)


#### Cross validation

In [40]:
# Do a 10 fold cross validation as that is done typically.
hpr_cross_val = H2ORandomForestEstimator(model_id='housing_price_regression', 
                                         seed=1, 
                                         nfolds=5,
                                         keep_cross_validation_predictions=True)
hpr_cross_val.train(x=all_cols_in_model, 
                    y=dep_var_col, 
                    training_frame=get_h2o_frame_with_rel_factors(train_validation_data))


Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%


In [41]:
hpr_cross_val.cross_validation_predictions

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  housing_price_regression


ModelMetricsRegression: drf
** Reported on train data. **

MSE: 0.019799110697422875
RMSE: 0.1407093127601115
MAE: 0.09757718828825032
RMSLE: 0.01093486090327993
Mean Residual Deviance: 0.019799110697422875

ModelMetricsRegression: drf
** Reported on cross-validation data. **

MSE: 0.01938395105795873
RMSE: 0.13922625850736178
MAE: 0.09577830267605716
RMSLE: 0.01083603165391223
Mean Residual Deviance: 0.01938395105795873
Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,0.0956881,0.0058530,0.1084217,0.0829694,0.0954006,0.0988700,0.0927787
mean_residual_deviance,0.0193422,0.0029547,0.0252202,0.0125499,0.0191893,0.0216115,0.0181401
mse,0.0193422,0.0029547,0.0252202,0.0125499,0.0191893,0.0216115,0.0181401
r2,0.8790135,0.0117323,0.8703818,0.8985824,0.8720775,0.8562414,0.8977843
residual_deviance,0.0193422,0.0029547,0.0252202,0.0125499,0.0191893,0.0216115,0.0181401
rmse,0.1382107,0.0109537,0.1588085,0.1120262,0.1385254,0.1470084,0.134685
rmsle,0.0107523,0.0008834,0.0123688,0.0086034,0.0107532,0.0114827,0.0105534


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
,2019-04-09 09:51:37,1.522 sec,0.0,nan,nan,nan
,2019-04-09 09:51:37,1.537 sec,1.0,0.1954341,0.1397954,0.0381945
,2019-04-09 09:51:37,1.548 sec,2.0,0.1908295,0.1352168,0.0364159
,2019-04-09 09:51:37,1.559 sec,3.0,0.1815573,0.1314005,0.0329631
,2019-04-09 09:51:37,1.570 sec,4.0,0.1760658,0.1279177,0.0309992
---,---,---,---,---,---,---
,2019-04-09 09:51:38,2.016 sec,46.0,0.1414475,0.0981634,0.0200074
,2019-04-09 09:51:38,2.027 sec,47.0,0.1413494,0.0980232,0.0199797
,2019-04-09 09:51:38,2.037 sec,48.0,0.1411098,0.0977282,0.0199120
,2019-04-09 09:51:38,2.047 sec,49.0,0.1408666,0.0975892,0.0198434



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
OverallQual,2756.4650879,1.0,0.3048628
Neighborhood,2168.5063477,0.7866983,0.2398350
GrLivArea,1208.7586670,0.4385177,0.1336877
TotalBsmtSF,714.8059692,0.2593198,0.0790570
GarageArea,542.7357788,0.1968956,0.0600261
1stFlrSF,459.9797363,0.1668730,0.0508734
YearRemodAdd,366.4526367,0.1329430,0.0405294
MSSubClass,345.7257690,0.1254236,0.0382370
BsmtFinSF1,175.9045105,0.0638153,0.0194549
2ndFlrSF,170.8649445,0.0619870,0.0188975


<bound method ModelBase.cross_validation_predictions of >

In [42]:
def get_cross_validated_rmse(hpr_cross_val):
    cv_preds = hpr_cross_val.cross_validation_predictions()
    for i in range(len(cv_preds)):
        if i == 0:
            result_cv = cv_preds[0]['predict'].as_data_frame().copy()
        else:
            result_cv +=  cv_preds[i]['predict'].as_data_frame()
    return evaluate_model_score_given_predictions(result_cv, train_validation_data['LogSalePrice'])    

In [43]:
get_cross_validated_rmse(hpr_cross_val)

0.13922626299526575

### Try somewhat of a greedy method to select columns

In this approach, we would start with an empty model and at every step, we would the predictor that would decrease the cross validation error the most. We have not fully automated it (it would be extremely easy to do so), but have built the model manually after inspection in each step. 

As simple as it seems, this looks to work pretty well. In fact, we were able to get a much better model, with almost no extra effort !

In [80]:
import operator
def get_cross_val_scores_new_col(base_model_cols, 
                                 train_validation_data=train_validation_data,
                                 nfolds=5,
                                 dep_var_col='LogSalePrice'):
    columns_to_cross_val_score = dict()
    for col in ALL_COLUMNS:
        # If the column was already included, skip it.
        if col in base_model_cols:
            continue
        
        cur_model_cols = base_model_cols + [col]            
        print(cur_model_cols)

        # Do a 10 fold cross validation as that is done typically.
        hpr_cross_val = H2ORandomForestEstimator(model_id='housing_price_regression', 
                                                 seed=1, 
                                                 nfolds=nfolds,
                                                 keep_cross_validation_predictions=True)
        hpr_cross_val.train(x=cur_model_cols, 
                            y=dep_var_col, 
                            training_frame=get_h2o_frame_with_rel_factors(train_validation_data))

        cv_score = get_cross_validated_rmse(hpr_cross_val)

        columns_to_cross_val_score[col] = cv_score
    
    sorted_cross_val_scores = sorted(columns_to_cross_val_score.items(), key=operator.itemgetter(1))
    return sorted_cross_val_scores

In [82]:
def get_cross_val_scores_given_cols(cur_model_cols,
                                    train_validation_data=train_validation_data,
                                    nfolds=5,
                                    dep_var_col='LogSalePrice'):

    # Do a 10 fold cross validation as that is done typically.
    hpr_cross_val = H2ORandomForestEstimator(model_id='housing_price_regression', 
                                             seed=1, 
                                             nfolds=nfolds,
                                             keep_cross_validation_predictions=True)
    hpr_cross_val.train(x=cur_model_cols, 
                        y=dep_var_col, 
                        training_frame=get_h2o_frame_with_rel_factors(train_validation_data))

    cv_score = get_cross_validated_rmse(hpr_cross_val)

    return cv_score

In [48]:
#sorted_cross_val_scores = get_cross_val_scores_new_col([])

['MSSubClass']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['MSZoning']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['Street']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['LotShape']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['LandContour']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['Utilities']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |

In [50]:
#sorted_cross_val_scores

[('OverallQual', 0.22982421832125344),
 ('Neighborhood', 0.2661876508712513),
 ('GarageCars', 0.2863561789819541),
 ('ExterQual', 0.2941283886407502),
 ('BsmtQual', 0.29631406694885337),
 ('KitchenQual', 0.2975537585280438),
 ('GarageArea', 0.2992157116026678),
 ('YearBuilt', 0.31028553626759886),
 ('GrLivArea', 0.3110556632966484),
 ('GarageFinish', 0.3151021633979988),
 ('GarageYrBlt_times_not_missing', 0.31954443950602),
 ('FullBath', 0.32034100136445326),
 ('GarageType', 0.3275768023883375),
 ('YearRemodAdd', 0.3304578802248884),
 ('MSSubClass', 0.3304683876637387),
 ('FireplaceQu', 0.3340680181155671),
 ('Foundation', 0.3348166520747213),
 ('TotRmsAbvGrd', 0.33798477079298456),
 ('TotalBsmtSF', 0.3416650172863745),
 ('Fireplaces', 0.34385830432543923),
 ('HeatingQC', 0.350979295322223),
 ('BsmtFinType1', 0.358082663943684),
 ('1stFlrSF', 0.3586992985754942),
 ('MasVnrType', 0.3597555919081454),
 ('MSZoning', 0.3637349598674163),
 ('2ndFlrSF', 0.3661242254067586),
 ('Exterior1st', 

In [51]:
#sorted_cross_val_scores = get_cross_val_scores_new_col(['OverallQual'])

['OverallQual', 'MSSubClass']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'MSZoning']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Street']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'LotShape']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'LandContour']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Utilities']
Parse progress: |█

In [52]:
#sorted_cross_val_scores

[('Neighborhood', 0.20165097982404817),
 ('GarageCars', 0.2098824734041076),
 ('GrLivArea', 0.2156129292993399),
 ('MSSubClass', 0.2158233402444538),
 ('TotRmsAbvGrd', 0.21702584917585035),
 ('Fireplaces', 0.218986207375074),
 ('FullBath', 0.22119400344995407),
 ('BedroomAbvGr', 0.22224915909070095),
 ('GarageType', 0.2230369642575545),
 ('FireplaceQu', 0.22498062287259563),
 ('BsmtFullBath', 0.22545587180430332),
 ('GarageFinish', 0.22571513251959147),
 ('YearRemodAdd', 0.22678274459020362),
 ('HalfBath', 0.22679020341463504),
 ('GarageArea', 0.2270247276064446),
 ('LotArea', 0.22741356253093556),
 ('2ndFlrSF', 0.22866265700480773),
 ('OverallQual', 0.22982421832125344),
 ('MSZoning', 0.2299674112381472),
 ('GarageYrBlt_times_not_missing', 0.23000796885642677),
 ('KitchenQual', 0.23004958263303413),
 ('BsmtQual', 0.2300613039043453),
 ('Exterior1st', 0.23039055460003005),
 ('BsmtHalfBath', 0.2317269596013994),
 ('Exterior2nd', 0.231885643361254),
 ('OverallCond', 0.232054100057026),
 

In [53]:
#sorted_cross_val_scores = get_cross_val_scores_new_col(['OverallQual', 'Neighborhood'])

['OverallQual', 'Neighborhood', 'MSSubClass']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'MSZoning']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'Street']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'LotShape']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'LandContour']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |█████████████████████

In [54]:
#sorted_cross_val_scores


[('GrLivArea', 0.17181073621260148),
 ('1stFlrSF', 0.18468465535273326),
 ('2ndFlrSF', 0.18881612747684262),
 ('LotArea', 0.18975826420145564),
 ('GarageArea', 0.19083247056983663),
 ('TotalBsmtSF', 0.19100328473519682),
 ('GarageCars', 0.19217711808107713),
 ('MSSubClass', 0.19291267041881155),
 ('YearRemodAdd', 0.1957059220835412),
 ('GarageYrBlt_times_not_missing', 0.19577735096207094),
 ('TotRmsAbvGrd', 0.19594176245492187),
 ('Fireplaces', 0.1963133312946919),
 ('BsmtFinSF1', 0.1963521652286718),
 ('FireplaceQu', 0.19723350867589046),
 ('FullBath', 0.19847767178432504),
 ('GarageType', 0.19862321644113773),
 ('KitchenQual', 0.19869210046630717),
 ('MasVnrArea_times_not_missing', 0.19926446625546967),
 ('GarageFinish', 0.19946582277862537),
 ('SaleType', 0.20031945449327093),
 ('BsmtExposure', 0.2005769019888137),
 ('YearBuilt', 0.20079843435621578),
 ('SaleCondition', 0.20110668458206396),
 ('Exterior1st', 0.20112311645073414),
 ('BedroomAbvGr', 0.20120612238403116),
 ('WoodDeckSF

In [55]:
#sorted_cross_val_scores = get_cross_val_scores_new_col(['OverallQual', 'Neighborhood', 'GrLivArea'])

['OverallQual', 'Neighborhood', 'GrLivArea', 'MSSubClass']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'MSZoning']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'Street']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'LotShape']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'LandContour']
Parse progress: |███████████████████████████████████████████████

drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', '3SsnPorch']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'ScreenPorch']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'PoolArea']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'MiscVal']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%


In [56]:
#sorted_cross_val_scores

[('BsmtFinSF1', 0.15584090695194738),
 ('TotalBsmtSF', 0.16047608458931067),
 ('YearRemodAdd', 0.16049577598888637),
 ('2ndFlrSF', 0.1614275881845015),
 ('1stFlrSF', 0.16207537987737147),
 ('BsmtFinType1', 0.1621668729541667),
 ('GarageYrBlt_times_not_missing', 0.1628690855862338),
 ('GarageArea', 0.1628799891256461),
 ('YearBuilt', 0.1640216413848401),
 ('OverallCond', 0.16411340422175816),
 ('HouseStyle', 0.16452627022817382),
 ('MSSubClass', 0.1645730266034459),
 ('GarageCars', 0.16465936734426395),
 ('BsmtQual', 0.16472457201747934),
 ('LotArea', 0.16496818833803484),
 ('BsmtExposure', 0.16582857233133372),
 ('Fireplaces', 0.16595671500247525),
 ('BsmtUnfSF', 0.16607123227125087),
 ('BsmtFullBath', 0.16695154395875528),
 ('GarageType', 0.16703143281007277),
 ('GarageFinish', 0.16715815757000635),
 ('FireplaceQu', 0.16720653322128406),
 ('KitchenQual', 0.16805836121363982),
 ('WoodDeckSF', 0.16812183392770255),
 ('BsmtCond', 0.16819827740353083),
 ('BedroomAbvGr', 0.1683136433746423

In [57]:
#sorted_cross_val_scores = get_cross_val_scores_new_col(['OverallQual', 
#                                                        'Neighborhood', 
#                                                        'GrLivArea',
#                                                        'BsmtFinSF1'])

['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'MSSubClass']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'MSZoning']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'Street']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'LotShape']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'LandCont

['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageArea']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'WoodDeckSF']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'OpenPorchSF']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 

In [58]:
#sorted_cross_val_scores

[('GarageCars', 0.15165532661189268),
 ('GarageArea', 0.1517788549307199),
 ('GarageYrBlt_times_not_missing', 0.151865497801642),
 ('YearRemodAdd', 0.15272092493532527),
 ('OverallCond', 0.15371758728508306),
 ('LotArea', 0.15471700944196973),
 ('1stFlrSF', 0.15495882561970997),
 ('TotalBsmtSF', 0.1554599423485044),
 ('MSSubClass', 0.1555002610519521),
 ('YearBuilt', 0.15571177213868073),
 ('Neighborhood', 0.15584090695194738),
 ('OverallQual', 0.15584090695194738),
 ('BsmtFinSF1', 0.15584090695194738),
 ('GrLivArea', 0.15584090695194738),
 ('HalfBath', 0.15595064654442894),
 ('Fireplaces', 0.15628391735478506),
 ('SaleType', 0.15635163274104327),
 ('KitchenQual', 0.15641468503176867),
 ('GarageType', 0.156451670887699),
 ('BedroomAbvGr', 0.15661921106065854),
 ('BldgType', 0.156706504015985),
 ('2ndFlrSF', 0.15671758533351082),
 ('HouseStyle', 0.15674574505545524),
 ('WoodDeckSF', 0.1569083287187503),
 ('GarageFinish', 0.15697857096146464),
 ('BsmtUnfSF', 0.15736743597262984),
 ('Kitc

In [59]:
'''
sorted_cross_val_scores = get_cross_val_scores_new_col(['OverallQual', 
                                                        'Neighborhood', 
                                                        'GrLivArea',
                                                        'BsmtFinSF1',
                                                        'GarageCars'])
'''

['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars', 'MSSubClass']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars', 'MSZoning']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars', 'Street']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars', 'LotShape']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQua

Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars', 'Fireplaces']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars', 'GarageYrBlt_times_not_missing']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars', 'GarageYrBlt_not_missing']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 

In [61]:
#sorted_cross_val_scores

[('OverallCond', 0.14502188441537864),
 ('YearRemodAdd', 0.14628606253495904),
 ('HeatingQC', 0.1472393531571203),
 ('1stFlrSF', 0.14820151250293345),
 ('LotArea', 0.1483945319494149),
 ('FireplaceQu', 0.14884540525284187),
 ('BldgType', 0.1489853328528062),
 ('2ndFlrSF', 0.1490834911975267),
 ('Fireplaces', 0.1493498056790633),
 ('CentralAir', 0.14935839262242676),
 ('HouseStyle', 0.149564736662932),
 ('KitchenQual', 0.14957246136146995),
 ('BsmtCond', 0.14962117638786288),
 ('YearBuilt', 0.14981062312284782),
 ('MSSubClass', 0.1499008461518045),
 ('SaleCondition', 0.14993509566239568),
 ('GarageFinish', 0.15006917369307068),
 ('LotConfig', 0.15013523783865684),
 ('TotalBsmtSF', 0.15021239990007934),
 ('HalfBath', 0.1502168276490194),
 ('BsmtQual', 0.15022725079345445),
 ('WoodDeckSF', 0.15026272269317095),
 ('BedroomAbvGr', 0.1502777147332396),
 ('PavedDrive', 0.1503957621246963),
 ('GarageYrBlt_times_not_missing', 0.15040106998924566),
 ('BsmtExposure', 0.15042761644617794),
 ('Exte

In [62]:
'''
sorted_cross_val_scores = get_cross_val_scores_new_col(['OverallQual', 
                                                        'Neighborhood', 
                                                        'GrLivArea',
                                                        'BsmtFinSF1',
                                                        'GarageCars',
                                                        'OverallCond'])
'''

['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars', 'OverallCond', 'MSSubClass']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars', 'OverallCond', 'MSZoning']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars', 'OverallCond', 'Street']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars', 'OverallCond', 'LotShape']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |██████

['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars', 'OverallCond', 'HalfBath']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars', 'OverallCond', 'BedroomAbvGr']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars', 'OverallCond', 'KitchenAbvGr']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars', 'OverallCond', 'TotRmsAbvGrd']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progr

In [63]:
#sorted_cross_val_scores

[('YearBuilt', 0.14087625516859886),
 ('TotalBsmtSF', 0.14250958276710543),
 ('Fireplaces', 0.14284243608882072),
 ('YearRemodAdd', 0.1432004998607045),
 ('SaleCondition', 0.1432293687816132),
 ('SaleType', 0.14326662900109527),
 ('BsmtQual', 0.14371764783158905),
 ('HeatingQC', 0.14375681079368544),
 ('1stFlrSF', 0.14386569697489027),
 ('FireplaceQu', 0.1439049036559225),
 ('2ndFlrSF', 0.14401686452551068),
 ('KitchenAbvGr', 0.14403321770880803),
 ('MSSubClass', 0.14405759163416548),
 ('GarageYrBlt_times_not_missing', 0.1441353822322847),
 ('PavedDrive', 0.14413820674405697),
 ('BsmtFullBath', 0.14418929326638844),
 ('LotArea', 0.14424117039584644),
 ('CentralAir', 0.14439666994955547),
 ('BsmtUnfSF', 0.1444724582219141),
 ('MSZoning', 0.14457545862221688),
 ('HalfBath', 0.14465424875839525),
 ('BldgType', 0.14469674208145739),
 ('WoodDeckSF', 0.14478699408116077),
 ('Condition1', 0.14479677637266355),
 ('BedroomAbvGr', 0.14484590633003916),
 ('HouseStyle', 0.1448819349643838),
 ('Bsm

In [64]:
'''
sorted_cross_val_scores = get_cross_val_scores_new_col(['OverallQual', 
                                                        'Neighborhood', 
                                                        'GrLivArea',
                                                        'BsmtFinSF1',
                                                        'GarageCars',
                                                        'OverallCond',
                                                        'YearBuilt'])
'''

['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars', 'OverallCond', 'YearBuilt', 'MSSubClass']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars', 'OverallCond', 'YearBuilt', 'MSZoning']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars', 'OverallCond', 'YearBuilt', 'Street']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars', 'OverallCond', 'YearBuilt', 'LotShape']
Parse progress: |█████████████████████████████████████████████

['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars', 'OverallCond', 'YearBuilt', 'BsmtFullBath']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars', 'OverallCond', 'YearBuilt', 'BsmtHalfBath']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars', 'OverallCond', 'YearBuilt', 'FullBath']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars', 'OverallCond', 'YearBuilt', 'HalfBath']
Parse progress: |█████████████████████████████████████

In [65]:
#sorted_cross_val_scores

[('LotArea', 0.13831687965092432),
 ('Fireplaces', 0.13886863073942746),
 ('FireplaceQu', 0.13918549434327546),
 ('TotalBsmtSF', 0.13956949676730793),
 ('MSZoning', 0.1395864288447212),
 ('BldgType', 0.14001908411039807),
 ('2ndFlrSF', 0.14008952415555145),
 ('LandContour', 0.1401251849979675),
 ('MSSubClass', 0.1402981763142285),
 ('BsmtExposure', 0.14031436329054978),
 ('KitchenQual', 0.1404157217556968),
 ('YearRemodAdd', 0.14046606209952545),
 ('CentralAir', 0.14051680806114483),
 ('BsmtQual', 0.14065749100855127),
 ('Utilities', 0.14069721904848567),
 ('ExterCond', 0.14073652630766537),
 ('HeatingQC', 0.14080899124273782),
 ('Neighborhood', 0.14087625516859886),
 ('OverallQual', 0.14087625516859886),
 ('OverallCond', 0.14087625516859886),
 ('YearBuilt', 0.14087625516859886),
 ('BsmtFinSF1', 0.14087625516859886),
 ('GrLivArea', 0.14087625516859886),
 ('GarageCars', 0.14087625516859886),
 ('KitchenAbvGr', 0.14091003488563905),
 ('LowQualFinSF', 0.14097590874580324),
 ('BsmtUnfSF', 0

In [66]:
'''
sorted_cross_val_scores = get_cross_val_scores_new_col(['OverallQual', 
                                                        'Neighborhood', 
                                                        'GrLivArea',
                                                        'BsmtFinSF1',
                                                        'GarageCars',
                                                        'OverallCond',
                                                        'YearBuilt',
                                                        'LotArea'])
'''

['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars', 'OverallCond', 'YearBuilt', 'LotArea', 'MSSubClass']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars', 'OverallCond', 'YearBuilt', 'LotArea', 'MSZoning']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars', 'OverallCond', 'YearBuilt', 'LotArea', 'Street']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars', 'OverallCond', 'YearBuilt', 'LotArea', 'LotShape']
Parse progress: |█

['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars', 'OverallCond', 'YearBuilt', 'LotArea', 'LowQualFinSF']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars', 'OverallCond', 'YearBuilt', 'LotArea']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars', 'OverallCond', 'YearBuilt', 'LotArea', 'BsmtFullBath']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars', 'OverallCond', 'YearBuilt', 'LotArea', 'BsmtHalfBath']
Parse progress: |█

In [67]:
sorted_cross_val_scores

[('Fireplaces', 0.1353422118898401),
 ('FireplaceQu', 0.1362783328387146),
 ('TotalBsmtSF', 0.1364903319196077),
 ('CentralAir', 0.1367277840780513),
 ('PavedDrive', 0.13682039860113548),
 ('SaleCondition', 0.1368504194820313),
 ('SaleType', 0.13695721858249196),
 ('GarageCond', 0.1371601488262971),
 ('Fence', 0.13720969102626618),
 ('PoolArea', 0.13721084418630625),
 ('ExterCond', 0.13722172661878282),
 ('PoolQC', 0.13745434347692903),
 ('WoodDeckSF', 0.13750395136670607),
 ('KitchenQual', 0.1375358304502706),
 ('BldgType', 0.1375429784987448),
 ('BsmtExposure', 0.13754889529751568),
 ('KitchenAbvGr', 0.13767816404417818),
 ('GarageQual', 0.13768133198839594),
 ('3SsnPorch', 0.13778170601608317),
 ('MasVnrArea_not_missing', 0.13781965176830213),
 ('Condition2', 0.13783645326621363),
 ('MiscFeature', 0.13786414848136203),
 ('GarageYrBlt_not_missing', 0.13791793263398003),
 ('MasVnrType', 0.13792898060362255),
 ('BsmtCond', 0.1379581795661578),
 ('OpenPorchSF', 0.13796626467775608),
 ('

In [71]:
'''
sorted_cross_val_scores = get_cross_val_scores_new_col(['OverallQual', 
                                                        'Neighborhood', 
                                                        'GrLivArea',
                                                        'BsmtFinSF1',
                                                        'GarageCars',
                                                        'OverallCond',
                                                        'YearBuilt',
                                                        'LotArea',
                                                        'Fireplaces'])
'''


"\nsorted_cross_val_scores = get_cross_val_scores_new_col(['OverallQual', \n                                                        'Neighborhood', \n                                                        'GrLivArea',\n                                                        'BsmtFinSF1',\n                                                        'GarageCars',\n                                                        'OverallCond',\n                                                        'YearBuilt',\n                                                        'LotArea',\n                                                        'Fireplaces'])\n"

In [69]:
#sorted_cross_val_scores

[('CentralAir', 0.13386413014950854),
 ('LandContour', 0.13444115592845524),
 ('ExterCond', 0.13524731646413182),
 ('Neighborhood', 0.1353422118898401),
 ('LotArea', 0.1353422118898401),
 ('OverallQual', 0.1353422118898401),
 ('OverallCond', 0.1353422118898401),
 ('YearBuilt', 0.1353422118898401),
 ('BsmtFinSF1', 0.1353422118898401),
 ('GrLivArea', 0.1353422118898401),
 ('Fireplaces', 0.1353422118898401),
 ('GarageCars', 0.1353422118898401),
 ('MSZoning', 0.13537268733276286),
 ('PavedDrive', 0.13544209930635726),
 ('Street', 0.13546060236910137),
 ('SaleType', 0.13551084389390167),
 ('LotConfig', 0.13551416544426667),
 ('ScreenPorch', 0.13552828328311775),
 ('PoolArea', 0.13564567518626822),
 ('PoolQC', 0.135665386275448),
 ('Foundation', 0.13571904322853243),
 ('KitchenQual', 0.13573223872720674),
 ('MiscFeature', 0.1357398416683308),
 ('HeatingQC', 0.13577916640212725),
 ('BsmtQual', 0.13578905964780702),
 ('SaleCondition', 0.1358353673191613),
 ('2ndFlrSF', 0.13585628128919677),
 (

In [72]:
'''
sorted_cross_val_scores = get_cross_val_scores_new_col(['OverallQual', 
                                                        'Neighborhood', 
                                                        'GrLivArea',
                                                        'BsmtFinSF1',
                                                        'GarageCars',
                                                        'OverallCond',
                                                        'YearBuilt',
                                                        'LotArea',
                                                        'Fireplaces',
                                                        'CentralAir'])
'''


['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars', 'OverallCond', 'YearBuilt', 'LotArea', 'Fireplaces', 'CentralAir', 'MSSubClass']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars', 'OverallCond', 'YearBuilt', 'LotArea', 'Fireplaces', 'CentralAir', 'MSZoning']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars', 'OverallCond', 'YearBuilt', 'LotArea', 'Fireplaces', 'CentralAir', 'Street']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1',

['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars', 'OverallCond', 'YearBuilt', 'LotArea', 'Fireplaces', 'CentralAir']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars', 'OverallCond', 'YearBuilt', 'LotArea', 'Fireplaces', 'CentralAir', 'BsmtUnfSF']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars', 'OverallCond', 'YearBuilt', 'LotArea', 'Fireplaces', 'CentralAir', 'TotalBsmtSF']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'Garage

In [73]:
#sorted_cross_val_scores

[('ExterCond', 0.13318037477953817),
 ('MSZoning', 0.13368685278814144),
 ('BsmtExposure', 0.13376857602833217),
 ('Neighborhood', 0.13386413014950854),
 ('CentralAir', 0.13386413014950854),
 ('LotArea', 0.13386413014950854),
 ('OverallQual', 0.13386413014950854),
 ('OverallCond', 0.13386413014950854),
 ('YearBuilt', 0.13386413014950854),
 ('BsmtFinSF1', 0.13386413014950854),
 ('GrLivArea', 0.13386413014950854),
 ('Fireplaces', 0.13386413014950854),
 ('GarageCars', 0.13386413014950854),
 ('LowQualFinSF', 0.13398326178046185),
 ('TotalBsmtSF', 0.13400649672935067),
 ('BldgType', 0.13426506076554873),
 ('SaleCondition', 0.13447708908709594),
 ('PavedDrive', 0.13453169238046342),
 ('SaleType', 0.13453657792867535),
 ('3SsnPorch', 0.1346002444699165),
 ('BsmtCond', 0.1346323561795224),
 ('HalfBath', 0.134640168682726),
 ('BsmtFinType2', 0.1346596659285976),
 ('MasVnrType', 0.13468285970346325),
 ('ScreenPorch', 0.1346888743935755),
 ('KitchenQual', 0.13473959682539222),
 ('Condition1', 0.1

In [78]:
sorted_cross_val_scores = get_cross_val_scores_new_col(['OverallQual', 
                                                        'Neighborhood', 
                                                         'GrLivArea',
                                                        'BsmtFinSF1',
                                                        'GarageCars',
                                                        'OverallCond',
                                                        'YearBuilt',
                                                        'LotArea',
                                                        'Fireplaces',
                                                        'CentralAir',
                                                        'ExterCond'])



['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars', 'OverallCond', 'YearBuilt', 'LotArea', 'Fireplaces', 'CentralAir', 'ExterCond', 'MSSubClass']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars', 'OverallCond', 'YearBuilt', 'LotArea', 'Fireplaces', 'CentralAir', 'ExterCond', 'MSZoning']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars', 'OverallCond', 'YearBuilt', 'LotArea', 'Fireplaces', 'CentralAir', 'ExterCond', 'Street']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Ne

Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars', 'OverallCond', 'YearBuilt', 'LotArea', 'Fireplaces', 'CentralAir', 'ExterCond', 'BsmtFullBath']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars', 'OverallCond', 'YearBuilt', 'LotArea', 'Fireplaces', 'CentralAir', 'ExterCond', 'BsmtHalfBath']
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
['OverallQual', 'Neighborhood', 'GrLivArea', 'BsmtFinSF1', 'GarageCars', 'OverallCond', 'YearBuilt', 'LotArea', 'Fireplaces', 'CentralAir', 'ExterCond', 'FullBath']
Parse progr

In [79]:
sorted_cross_val_scores

[('YearRemodAdd', 0.13357889753316388),
 ('KitchenAbvGr', 0.13424023975099275),
 ('MSZoning', 0.1342506213873648),
 ('RoofMatl', 0.1343139636615266),
 ('2ndFlrSF', 0.13442283683669068),
 ('BldgType', 0.1344909458381618),
 ('MasVnrType', 0.13451367825811059),
 ('BsmtQual', 0.13457373111725413),
 ('KitchenQual', 0.13458859326117534),
 ('SaleCondition', 0.13458906806114015),
 ('BsmtFinType1', 0.13466932297464246),
 ('MiscFeature', 0.13471533408053524),
 ('BsmtExposure', 0.13472850803877107),
 ('3SsnPorch', 0.1347326983586935),
 ('PavedDrive', 0.1347579904448328),
 ('HalfBath', 0.1347948213206885),
 ('ExterQual', 0.13484117607853957),
 ('BedroomAbvGr', 0.13486481745263615),
 ('Functional', 0.13490828517001419),
 ('LowQualFinSF', 0.1349971910633761),
 ('TotalBsmtSF', 0.13501259407102242),
 ('Condition2', 0.1350321314567296),
 ('GarageYrBlt_not_missing', 0.135068976865008),
 ('PoolQC', 0.13509011638833593),
 ('BsmtUnfSF', 0.13509325551294343),
 ('GarageCond', 0.13510946370069643),
 ('Conditi

We see that the cross validation score is decreasing, indicating that we have reached a plateau. Let us make check out our cross validation score now.

In [104]:
final_cols = ['OverallQual', 
              'Neighborhood', 
              'GrLivArea', 
              'BsmtFinSF1',
              'GarageCars',
              'OverallCond',
              'YearBuilt',
              'LotArea',
              'Fireplaces',
              'CentralAir',
              'ExterCond']

In [105]:
cv_score = get_cross_val_scores_given_cols(final_cols)
print(cv_score)


Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
0.13318037477953817


### Quantifying validation error on validation data set.

Ideally this is not necessary as we have already performed cross validation over the entire training data set, but it is still useful to do it as this be a comparable yardstick against other models and can be useful while ensembling as well.

In [106]:
# Do a 10 fold cross validation as that is done typically.
model_train_data = H2ORandomForestEstimator(model_id='housing_price_regression', 
                                            seed=1)
model_train_data.train(x=final_cols, 
                       y=dep_var_col, 
                       training_frame=get_h2o_frame_with_rel_factors(train_data))

predict_validation = model_train_data.predict(
    get_h2o_frame_with_rel_factors(validation_data))                

validation_data['Predictions'] = predict_validation['predict'].as_data_frame()
evaluate_model_score_given_predictions(validation_data['LogSalePrice'], validation_data['Predictions'])


Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%


0.12942634553451818

### Making predictions on test data

Now, there is another important matter to consider. In order to make predictions on test data, firstly it is important that we make sure that the test data is good. 

One problem that could be relevant here is presence of lots of null values in the test set and not in the training set. Let us check whether that is the case

In [87]:
test_data[final_cols].isnull().sum()

OverallQual     0
Neighborhood    0
GrLivArea       0
BsmtFinSF1      1
GarageCars      1
OverallCond     0
YearBuilt       0
LotArea         0
Fireplaces      0
CentralAir      0
ExterCond       0
dtype: int64

Okay, things look good. Let us generate predictions on the test set.

In [88]:
final_model = H2ORandomForestEstimator(model_id='housing_price_regression', seed=1)
final_model.train(x=final_cols, 
                  y=dep_var_col, 
                  training_frame=get_h2o_frame_with_rel_factors(train_validation_data))


Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%


In [89]:
predict_out = final_model.predict(
    get_h2o_frame_with_rel_factors(test_data))                


Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%


In [95]:
test_data['LogSalePrice'] = predict_out['predict'].as_data_frame()

In [97]:
test_data['SalePrice'] = test_data['LogSalePrice'].apply(lambda x : np.exp(x))

In [98]:
test_data[['Id', 'SalePrice']].to_csv('housing_price_random_forest_predictions.csv', index=False)